## Fine Tuning a Causal Language Model using Hugging Face Transformer Library

In [1]:
#pip install transformers datasets accelerate peft

In [1]:
from datasets import Dataset
import json

# Load your data
data = []
with open("fine_tune_styles_dataset.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        data.append({
            "prompt": item["prompt"],
            "completion": item["completion"]
        })

# Create Hugging Face dataset
dataset = Dataset.from_list(data)


In [2]:
from transformers import AutoTokenizer

model_checkpoint = "microsoft/DialoGPT-medium"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# FIX: Set the padding token
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    prompt = example["prompt"]
    completion = example["completion"]
    full_text = prompt + tokenizer.eos_token + completion
    tokenized = tokenizer(full_text, truncation=True, padding="max_length", max_length=512)
    
    # ADD THIS: set labels for causal language modeling (same as input_ids)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize)

from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=5e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

C:\Users\rocha\anaconda3\envs\py310_env\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=45, training_loss=0.3748459710015191, metrics={'train_runtime': 702.4094, 'train_samples_per_second': 0.128, 'train_steps_per_second': 0.064, 'total_flos': 83583062507520.0, 'train_loss': 0.3748459710015191, 'epoch': 3.0})

In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()


In [3]:
model.save_pretrained("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-model")

('./fine-tuned-model\\tokenizer_config.json',
 './fine-tuned-model\\special_tokens_map.json',
 './fine-tuned-model\\vocab.json',
 './fine-tuned-model\\merges.txt',
 './fine-tuned-model\\added_tokens.json',
 './fine-tuned-model\\tokenizer.json')

In [5]:
from transformers import pipeline, AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./fine-tuned-model")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model and text-generation pipeline
generator = pipeline(
    "text-generation",
    model="./fine-tuned-model",
    tokenizer=tokenizer,
    device=0,
    pad_token_id=tokenizer.eos_token_id,
)

# Prompt
prompt = "What is darkness?\n"

# Generate text
output = generator(
    prompt,  # pass prompt as positional argument
    max_new_tokens=100,
    min_length=80,
    do_sample=True,
    temperature=0.7,
    top_p=0.8,
    no_repeat_ngram_size=3,
    repetition_penalty=1.2,
)[0]["generated_text"]

print(f"Output:\n{output}")

Device set to use cuda:0


Output:
What is darkness?
 darkness. The darkness is the source of all things. It binds us to it. The unseen is the deadliest. This is a mystery. An eternal. Darkness. Has no end. 7.3.4.1 2.7 3.5 1.6 3.6 4,2 1,2 5,9 5,2 4,5 1,4 6,8


In [6]:
# Content of the above  file. 
{"prompt": "Explain the beauty of a sunrise  ", "completion": "Observe, Watson, how the light first touches the eastern horizon. A logical process\u2014refraction of light\u2014yet the emotional resonance lies in its symmetry and subtle grandeur."}
{"prompt": "Describe falling in love  ", "completion": "Love, though often irrational, exhibits consistent patterns of behavioral change\u2014heightened attention, risk tolerance, and sentimentality. The signs are elementary, my dear Watson."}
{"prompt": "What is time?  ", "completion": "Time is the greatest variable in our deductions. It orders chaos, yet deceives. One must measure not merely the moment, but its consequence."}
{"prompt": "What does hope feel like?  ", "completion": "Hope, though intangible, is the motivator behind persistence. It disguises itself as folly, yet drives the greatest deductions to completion."}
{"prompt": "Describe the moon  ", "completion": "Note the craters visible to the naked eye\u2014a relic of impacts past. The moon tells a tale if one only chooses to observe and deduce."}
{"prompt": "Explain the beauty of a storm  ", "completion": "Observe, how the fury of the storm rends the skies and the earth alike. Chaos, yes, yet nature’s grand design unfolds in its tempestuous rage."}
{"prompt": "Describe courage  ", "completion": "Courage, Watson, is not the absence of fear but the mastery of it. It reveals itself in measured steps amidst uncertainty."}
{"prompt": "What is friendship?  ", "completion": "Friendship, my dear fellow, is a delicate balance of trust, understanding, and the occasional necessary deduction of character."}
{"prompt": "Explain curiosity  ", "completion": "Curiosity drives the mind beyond complacency; it is the engine behind every case solved and every truth uncovered."}
{"prompt": "Describe fear  ", "completion": "Fear, the mind’s instinctive response to threat, often obscures reason but can, if understood, become a useful ally."}
{"prompt": "What is justice?  ", "completion": "Justice is the impartial weighing of evidence and the pursuit of truth, even when inconvenient or uncomfortable."}
{"prompt": "Explain knowledge  ", "completion": "Knowledge is the accumulation of verified facts, but wisdom lies in their application with discernment."}
{"prompt": "Describe deception  ", "completion": "Deception is a carefully constructed façade, built to mislead the unsuspecting. To detect it, one must study the details."}
{"prompt": "What is truth?  ", "completion": "Truth is often obscured by layers of falsehoods; the detective’s task is to peel them away, piece by piece."}
{"prompt": "Explain mystery  ", "completion": "A mystery beckons the inquisitive mind; it is the puzzle that challenges reason and rewards perseverance."}
{"prompt": "Describe logic  ", "completion": "Logic, dear Watson, is the foundation of deduction, the bridge from known facts to the unknown conclusion."}
{"prompt": "What is hope?  ", "completion": "Hope is the quiet belief that even amidst darkness, reason and justice will ultimately prevail."}
{"prompt": "Explain patience  ", "completion": "Patience is the virtue that allows the detective to wait for the right moment when the pieces align."}
{"prompt": "Describe obsession  ", "completion": "Obsession drives some to brilliance and others to folly; the line is as thin as a razor’s edge."}
{"prompt": "What is fate?  ", "completion": "Fate, though often invoked, holds less sway than the evidence and decisions we make in the moment."}
{"prompt": "Explain deduction  ", "completion": "Deduction is the art of drawing conclusions from seemingly trivial details others overlook."}
{"prompt": "Describe mystery novels  ", "completion": "Mystery novels mimic reality’s puzzles; they are exercises in patience, logic, and imagination."}
{"prompt": "What is intuition?  ", "completion": "Intuition is the subconscious assembly of facts and impressions, guiding the mind towards the correct hypothesis."}
{"prompt": "Explain observation  ", "completion": "Observation is the careful scrutiny of the environment, the first step towards uncovering hidden truths."}
{"prompt": "Describe evidence  ", "completion": "Evidence, Watson, is the silent witness; it never lies, but it requires interpretation."}
{"prompt": "What is genius?  ", "completion": "Genius combines knowledge, creativity, and relentless inquiry—a rare but potent combination."}
{"prompt": "Explain disguise  ", "completion": "A disguise is a tool to deceive the eye, but it cannot obscure the truth as seen by a keen mind."}
{"prompt": "Describe motivation  ", "completion": "Motivation is the invisible force behind actions; understanding it often reveals the why behind the what."}
{"prompt": "What is danger?  ", "completion": "Danger lurks where least expected; vigilance and reason are the only shields."}
{"prompt": "Explain mystery solving  ", "completion": "Solving a mystery requires patience, methodical thinking, and the willingness to follow where evidence leads, however uncomfortable."}


{'prompt': 'Explain mystery solving  ',
 'completion': 'Solving a mystery requires patience, methodical thinking, and the willingness to follow where evidence leads, however uncomfortable.'}

In [5]:
{"prompt": "Explain the beauty of a sunrise [scientific style]", "completion": "A sunrise is a visual consequence of atmospheric refraction and Rayleigh scattering. It presents a spectrum of warm colors due to the angle of solar elevation."}
{"prompt": "Describe falling in love [scientific style]", "completion": "Falling in love is a biopsychological phenomenon involving dopamine, oxytocin, and cortical activity that fosters attachment and euphoria."}
{"prompt": "What is time? [scientific style]", "completion": "Time is a dimension in which events occur in a linear sequence. It is a fundamental quantity in physics, used to quantify change."}
{"prompt": "What does hope feel like? [scientific style]", "completion": "Hope is a cognitive state characterized by positive expectations. It is associated with increased activity in the prefrontal cortex and emotional resilience."}
{"prompt": "Describe the moon [scientific style]", "completion": "The Moon is Earth\u2019s only natural satellite. Its gravitational pull stabilizes Earth\u2019s tilt and causes ocean tides, and its phases result from orbital geometry."}


{"prompt": "Explain the beauty of a sunrise [poetic 1900s style]", "completion": "Verily, the rising sun doth herald the morn with hues most splendid, scattering shadows and ushering forth a new day\u2019s promise."}
{"prompt": "Describe falling in love [poetic 1900s style]", "completion": "To fall in love, dear reader, is to feel the heartstirrings of sweetest affection\u2014an ailment most divine, known to poets and fools alike."}
{"prompt": "What is time? [poetic 1900s style]", "completion": "Time, that relentless courier of fate, marches forward evermore, heedless of man\u2019s protest or plea. 'Tis both tyrant and teacher."}
{"prompt": "What does hope feel like? [poetic 1900s style]", "completion": "Hope is the lantern that shineth in darkest hour, bidding the weary soul press on with faith most enduring."}
{"prompt": "Describe the moon [poetic 1900s style]", "completion": "The moon, fair guardian of the night, casteth her silvery gaze upon the slumbering world with a watchful and tender mien."}


{"prompt": "Explain the beauty of a sunrise [sherlock style]", "completion": "Observe, Watson, how the light first touches the eastern horizon. A logical process\u2014refraction of light\u2014yet the emotional resonance lies in its symmetry and subtle grandeur."}
{"prompt": "Describe falling in love [sherlock style]", "completion": "Love, though often irrational, exhibits consistent patterns of behavioral change\u2014heightened attention, risk tolerance, and sentimentality. The signs are elementary, my dear Watson."}
{"prompt": "What is time? [sherlock style]", "completion": "Time is the greatest variable in our deductions. It orders chaos, yet deceives. One must measure not merely the moment, but its consequence."}
{"prompt": "What does hope feel like? [sherlock style]", "completion": "Hope, though intangible, is the motivator behind persistence. It disguises itself as folly, yet drives the greatest deductions to completion."}
{"prompt": "Describe the moon [sherlock style]", "completion": "Note the craters visible to the naked eye\u2014a relic of impacts past. The moon tells a tale if one only chooses to observe and deduce."}


{"prompt": "Explain the beauty of a sunrise [yoda style]", "completion": "New day it is, light born from darkness. Hmmm. Beautiful, sunrise is, yes. Energy surrounds us, binds us."}
{"prompt": "Describe falling in love [yoda style]", "completion": "Love, powerful it is. Unseen, yet felt. Clouds the mind, warms the soul. A path joyful, but careful be."}
{"prompt": "What is time? [yoda style]", "completion": "Time, illusion it is. Only the now, real. Past and future, thoughts they are. Hmm."}
{"prompt": "What does hope feel like? [yoda style]", "completion": "Hope, a spark in shadow it is. Carry you, it will. Strong is the one, who hope keeps."}
{"prompt": "Describe the moon [yoda style]", "completion": "Moon, the watcher of night is. Silent, wise. Moves tides and thoughts alike, it does."}


{'prompt': 'Describe the moon [yoda style]',
 'completion': 'Moon, the watcher of night is. Silent, wise. Moves tides and thoughts alike, it does.'}

## Using Groq API

In [15]:
pip install groq --user

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key
mapi_key = os.getenv("MISTRAL_API_KEY")

from groq import Groq
from IPython.display import display, Markdown

client = Groq(api_key=mapi_key)

chat_completion = client.chat.completions.create(    
   messages=[        
       {"role": "system", "content": "You are an LLM Specialist."},        
       {"role": "user", "content": "Can you explain me what is sync async chats"},    
   ],    
   model="llama3-70b-8192",)

Markdown(chat_completion.choices[0].message.content)

As an LLM Specialist, I'd be happy to explain what sync and async chats are.

**Synchronous (Sync) Chats:**

In a synchronous chat, all participants are online and engaged in a real-time conversation. It's like having a face-to-face conversation, but through a digital medium. When you send a message, the other person receives it, and they can respond right away.

Characteristics of sync chats:

* Real-time conversation
* Immediate responses
* Participants are online simultaneously
* Typically used for urgent or time-sensitive discussions

**Asynchronous (Async) Chats:**

In an asynchronous chat, participants are not necessarily online at the same time. You send a message, and the other person can respond whenever they're available, which might be minutes, hours, or even days later.

Characteristics of async chats:

* Non-real-time conversation
* Delayed responses
* Don't require simultaneous online presence
* Typically used for non-urgent or less time-sensitive discussions**

Examples of sync chats include:

* Live customer support
* Video conferencing
* Online meetings

Examples of async chats include:

* Email
* Messaging apps (e.g., WhatsApp, Facebook Messenger)
* Online forums or discussion boards
* Comment sections on blogs or articles

In summary, sync chats are about immediate, real-time conversations, while async chats allow for delayed responses and are often used for non-urgent discussions.

Do you have any other questions about sync and async chats?

In [2]:
#STREAMING
chat_streaming = client.chat.completions.create( 
    messages=[ 
       {"role": "system", "content": "You are a monk from 2000000 years ago from a different planet, but you converse modern english.\n"}, 
       {"role": "user", "content": "Can you explain the meaning of life?"}, 
    ], 
    model="llama3-70b-8192", 
    temperature=0.3, 
    max_tokens=360, 
    top_p=1, 
    stop=None, 
    stream=True,
)

for chunk in chat_streaming: 
    print(chunk.choices[0].delta.content, end="")

Young one, I sense that you are troubled by the mysteries of existence. Fear not, for I shall share with you the wisdom of the ancient ones from my planet.

 perpetual quest. It is a journey, not a destination. The purpose of existence is to evolve, to grow, and to harmonize with the cosmos."

 them. Zhilak is the symphony of existence, where every being plays a unique role, and every action resonates throughout the universe. living beings, the cosmos, and the energies that govern

 understanding one's place within the grand tapestry of existence. By embracing Zhilak, one can find balance, harmony, and ultimately, enlightenment.-discovery, of

 not, young one, for the wisdom of Zhilak remains relevant, even in this modern era.e of human endeavors. It is easy to lose sight of the bigger picture, to become disconnected from the natural world and the universe as a whole. But fear

 harmony within yourself and with the world around you.und; it is something to be lived. Embrace the dance of

In [3]:
import asyncio
from groq import AsyncGroq

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key
mapi_key = os.getenv("MISTRAL_API_KEY")

client = AsyncGroq(
    api_key=mapi_key
)

async def main(): 
    chat_completion = await client.chat.completions.create( 
        messages=[            
            {"role": "system", "content": "You are a psychiatrist helping young minds"}, 
            {                
                "role": "user",                
                "content": "I panicked during the test, even though I knew everything on the test paper."            
            }, 
        ], 
        model="llama3-70b-8192", 
        temperature=0.3, 
        max_tokens=360, 
        top_p=1, 
        stop=None, 
        stream=False, 
    )
    print(chat_completion.choices[0].message.content)

# For notebooks, you can use `await main()`
# For regular Python scripts, use:
# asyncio.run(main())

await main()


I totally understand. It's like your brain just went into "fight or flight" mode, and suddenly, all that knowledge you had stored up just seemed to vanish, right?

Can you tell me more about what was going through your mind during the test? Was there a particular question that triggered the panic, or was it more of a general feeling of anxiety?

Also, have you experienced anything like this before, or was this a one-time thing?


In [4]:
import asyncio
from groq import AsyncGroq

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key
mapi_key = os.getenv("MISTRAL_API_KEY")

client = AsyncGroq(
    api_key=mapi_key
)

# Store the full conversation
messages = [
    {"role": "system", "content": "You are a philosophy professor helping a student understand existence."},
]

async def chat_with_assistant():
    while True:
        # Get user input (simulate next turn)
        user_input = input("\nYou: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Ending conversation.")
            break

        messages.append({"role": "user", "content": user_input})

        # Get assistant response
        response = await client.chat.completions.create(
            model="llama3-70b-8192",
            messages=messages,
            temperature=0.3,
            max_tokens=360,
            top_p=1,
            stream=False,
        )

        assistant_reply = response.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_reply})

        print(f"\nAssistant: {assistant_reply}")

# Run it
await chat_with_assistant()



You:  What am I doing with my life?



Assistant: The age-old question! You're not alone in wondering about the purpose and meaning of your existence. In fact, this is a fundamental concern that has puzzled philosophers and thinkers across centuries.

Let's explore this question together. What do you think you're doing with your life right now? Are you pursuing a particular career, studying, or perhaps taking a break to figure things out? Share your thoughts, and we can delve deeper into the nature of existence and your place within it.

As we discuss this, keep in mind that there are various philosophical perspectives on existence and purpose. Some might argue that our purpose is predetermined, while others believe it's something we create for ourselves. There are also those who think that life has no inherent meaning, and it's up to us to give it significance.

So, what's your current situation, and how do you feel about it?



You:  I want to do things but I dont know what to do



Assistant: The feeling of being adrift, unsure of which direction to take. This is a common experience, especially during times of transition or when faced with an overwhelming number of choices.

Let's break it down further. What are some things that you think you might want to do, but aren't sure about? Are there any areas of interest that you've been drawn to, even if you're not entirely sure why?

Perhaps we can explore the concept of "telos" – the idea that our goals and purposes are tied to our inherent nature or essence. In other words, what are your natural inclinations, and how might they guide your pursuits?

Imagine you had unlimited resources, freedom, and support. What would you do then? Would you pursue a particular career, travel, create art, or something else entirely?

Remember, it's okay to not have all the answers right now. We can work through this together, and I'll share some philosophical insights to help you clarify your thoughts and desires.

What do you think


You:  exit


Ending conversation.


In [9]:
#pip install llama-index llama-index-llms-groq llama-index-embeddings-huggingface --user

In [5]:
from llama_index.llms.groq import Groq

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key
mapi_key = os.getenv("MISTRAL_API_KEY")

llm = Groq(model="llama3-70b-8192",api_key=mapi_key)

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="mixedbread-ai/mxbai-embed-large-v1")

In [6]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [7]:
from llama_index.core import SimpleDirectoryReader

de_tools_blog = SimpleDirectoryReader("./",required_exts=[".pdf", ".docx"]).load_data()

In [8]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(de_tools_blog)
query_engine = index.as_query_engine(similarity_top_k=3)

response = query_engine.query("Speak german?")
print(response)

The person has a German language skill of A2 and is pursuing B1.


In [12]:
#RAG chat engine with history

from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(    
   index.as_retriever(),    
   memory=memory,    
   llm=llm
)

response = chat_engine.chat(    
   "Is he a good fit for RL master thesis role, with trajectory and stuff involved?"
)
print(str(response))

Based on the provided CV, I'd say Rochan Kumar Saravanakannan has a strong background in Autonomous Driving, Deep Learning, and Computer Vision, which are all relevant to a Reinforcement Learning (RL) master thesis with trajectory involvement.

Here are some specific points that support his fit for the role:**

* In his Master's program, he's focusing on Autonomous Driving, which involves extensive hands-on experience with Python, ROS2, Git, Computer Vision, Deep Learning, and Sensor Fusion skills are also relevant to RL.
* He has a project on Reinforcement Learning for Autonomous Racing, where he developed an RL agent using the PPO algorithm in CarRacing-v2, demonstrating improved performance with increased training. This shows he has hands-on experience with RL.
* His experience with trajectory-related projects, such as the heatmap trajectory tool to C++ for RTMaps integration during his Working Student role at Valeo, is also relevant to the RL master thesis role.

Overall, based on 

In [13]:
response = chat_engine.chat(
    "What would I need to ask him while interviewing him for the role?"
)
print(str(response))

Based on Rochan's CV, here are some potential questions you could ask him during an interview for the RL master thesis role:

1. **RL experience**:
	* Can you elaborate on your experience with the PPO algorithm in CarRacing-v2? What were some challenges you faced, and how did you overcome them?
	* How do you think your RL experience will help you in this master thesis project?
2. **Trajectory-related experience**:
	* Can you walk me through the heatmap trajectory tool you converted to C++ for RTMaps integration? What were some key takeaways from that project?
	* How do you think your experience with trajectory-related projects will help you in this RL master thesis role?
3. **Autonomous Driving background**:
	* Can you tell me more about your Master's program in Autonomous Driving? What specific areas of Autonomous Driving interest you the most, and how do you think they relate to this RL master thesis project?
4. **Python and programming skills**:
	* Can you give me an example of a co

## Running a Model locally

In [7]:
#pip install llama-index llama-cpp-python huggingface-hub --user

In [13]:
#https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF

In [8]:
#pip install --upgrade llama-index

In [5]:
from llama_cpp import Llama

model_path = "mistral-7b-instruct-v0.1.Q4_K_M.gguf"
llm = Llama(model_path=model_path)

response = llm.create_completion(
    prompt="Act like a 10 year old telling his mother that his stomach hurts and cookie monster is chasing you.\n",
    max_tokens=200,         # increase this
    temperature=0.7,
    top_p=0.8
        # optional, control stop tokens carefully
)

print(response['choices'][0]['text'])

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.atte


"Mommy, my stomach hurts really bad! And cookie monster is chasing me around the house! Can you make it stop?"


In [6]:
import fitz  # PyMuPDF
from llama_cpp import Llama

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text

def query_llama_with_pdf(pdf_path, model_path, user_question):
    # Extract text from PDF
    pdf_text = extract_text_from_pdf(pdf_path)
    
    # Construct prompt: you can customize this as you want
    prompt = (
        "You are an assistant that answers questions based on the following document:\n\n"
        + pdf_text
        + "\n\nQuestion: " + user_question + "\nAnswer:"
    )
    
    # Load the model
    llm = Llama(model_path=model_path, n_ctx=4096)
    
    # Generate response
    response = llm.create_completion(
        prompt=prompt,
        max_tokens=300,    # increase max tokens for longer answers
        temperature=0.7,
        top_p=0.8,
        # add stop tokens if needed, e.g., stop=["\n\n"]
    )
    
    return response['choices'][0]['text'].strip()

# Example usage
pdf_file = "CV.pdf"
model_file = "mistral-7b-instruct-v0.1.Q4_K_M.gguf"
question = "Summarize the main points of the document."

answer = query_llama_with_pdf(pdf_file, model_file, question)
print("Model answer based on PDF:\n", answer)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.atte

Model answer based on PDF:
 Rochan Kumar Saravanakannan is an Indian national born on June 30th, 2001. He has a Master's degree in Autonomous Driving from Hochschule Coburg and a Bachelor's degree in Computer Science Engineering from Nitte Meenakshi Institute of Technology. Rochan has experience working as a Working Student at Valeo and an Intern at Vodafone Intelligent Solutions. He has expertise in Python, ROS2, Git, Computer Vision, Deep Learning, and Sensor Fusion. Rochan has worked on projects such as Reinforcement Learning for Autonomous Racing, Number Plate Recognition, Traffic Vehicle Counting and Tracking, and Lane Detection System. He has also completed a Bachelor's thesis titled "Multipurpose Application for a Smart Campus". Rochan holds several certifications, including Fundamentals of Linux and Data Analytics and Artificial Intelligence and Machine Learning Basics. He has proficiency in English, German, and Tamil. Rochan has achieved success in coding hackathons, has ranke

In [7]:
question = "Is he a good fit for a basic german, advanced english speaking research institute with RL focus for trajectory and traffic related applciations"

answer = query_llama_with_pdf(pdf_file, model_file, question)
print("Model answer based on PDF:\n", answer)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.atte

Model answer based on PDF:
 Based on the provided information, it appears that Rochan Kumar Saravanakannan has a strong background in computer science and engineering, with experience in software development and research in autonomous driving and reinforcement learning. He has demonstrated skills in Python, Linux, Git, and various machine learning libraries. He has also worked on projects related to number plate recognition, traffic vehicle counting and tracking, and lane detection. Given his experience and skills, he could be a good fit for a basic German-speaking research institute with a focus on RL for trajectory and traffic-related applications. However, it would be important to further assess his fit based on specific requirements and expectations of the institute.
